### Get info box (from Wikipedia)

In [1]:
from bs4 import BeautifulSoup as bs
import requests
!pip install scrapy-splash
!pip install Scrapy
import scrapy
from scrapy.selector import Selector
from scrapy_splash import SplashRequest

from urllib.request import urlopen

import html5lib

import time
from urllib.request import urlopen as uReq
from urllib.request import Request

In [2]:
#### Load web page
r= requests.get('https://en.wikipedia.org/wiki/Toy_Story_3')

### convert to a beautiful soup object
soup = bs(r.content)

#print out HTML

contents = soup.prettify()


In [3]:
info_box=soup.find(class_='infobox vevent')
print(info_box.prettify())


<table class="infobox vevent">
 <tbody>
  <tr>
   <th class="infobox-above summary" colspan="2" style="font-size: 125%; font-style: italic;">
    Toy Story 3
   </th>
  </tr>
  <tr>
   <td class="infobox-image" colspan="2">
    <a class="image" href="/wiki/File:Toy_Story_3_poster.jpg" title="All of the toys packed close together, holding up a large numeral '3', with Buzz, who is putting a friendly arm around Woody's shoulder, and Woody holding the top of the 3.">
     <img alt="All of the toys packed close together, holding up a large numeral '3', with Buzz, who is putting a friendly arm around Woody's shoulder, and Woody holding the top of the 3." class="thumbborder" data-file-height="326" data-file-width="220" decoding="async" height="326" src="//upload.wikimedia.org/wikipedia/en/6/69/Toy_Story_3_poster.jpg" width="220"/>
    </a>
    <div class="infobox-caption">
     Theatrical release poster
    </div>
   </td>
  </tr>
  <tr>
   <th class="infobox-label" scope="row" style="white-s

In [4]:
info_rows=info_box.find_all('tr')
for row in info_rows:
    print(row.prettify())

<tr>
 <th class="infobox-above summary" colspan="2" style="font-size: 125%; font-style: italic;">
  Toy Story 3
 </th>
</tr>

<tr>
 <td class="infobox-image" colspan="2">
  <a class="image" href="/wiki/File:Toy_Story_3_poster.jpg" title="All of the toys packed close together, holding up a large numeral '3', with Buzz, who is putting a friendly arm around Woody's shoulder, and Woody holding the top of the 3.">
   <img alt="All of the toys packed close together, holding up a large numeral '3', with Buzz, who is putting a friendly arm around Woody's shoulder, and Woody holding the top of the 3." class="thumbborder" data-file-height="326" data-file-width="220" decoding="async" height="326" src="//upload.wikimedia.org/wikipedia/en/6/69/Toy_Story_3_poster.jpg" width="220"/>
  </a>
  <div class="infobox-caption">
   Theatrical release poster
  </div>
 </td>
</tr>

<tr>
 <th class="infobox-label" scope="row" style="white-space: nowrap; padding-right: 0.65em;">
  Directed by
 </th>
 <td class="

In [38]:

def get_content_value(row_data):
    if row_data.find('li'):
        return [li.get_text(' ', strip=True).replace('\xa0', '') for li in row_data.find_all('li')] ##for all list in that row
    else:
        return row_data.get_text(' ', strip=True).replace('\xa0', '') ##tvarkome teksta su replace

movie_info = {} #empty dictionary

for index, row in enumerate(info_rows):
    if index == 0:
        movie_info['title']=row.find('th').get_text(' ', strip=True) ### kad išmti nereikalingus tarpus ir sujungtus zodzius
    elif index == 1:
        continue
    else:
        header = row.find('th')
        if header:
            content_key = row.find('th').get_text('', strip=True)
            content_value = get_content_value(row.find('td'))
            movie_info[content_key] = content_value
        
movie_info
    


{'title': 'Toy Story 3',
 'Directed by': 'Lee Unkrich',
 'Screenplay by': 'Michael Arndt',
 'Story by': ['John Lasseter', 'Andrew Stanton', 'Lee Unkrich'],
 'Produced by': 'Darla K. Anderson',
 'Starring': ['Tom Hanks',
  'Tim Allen',
  'Joan Cusack',
  'Don Rickles',
  'Wallace Shawn',
  'John Ratzenberger',
  'Estelle Harris',
  'Ned Beatty',
  'Michael Keaton',
  'Jodi Benson',
  'John Morris'],
 'Cinematography': ['Jeremy Lasky', 'Kim White'],
 'Edited by': 'Ken Schretzmann',
 'Music by': 'Randy Newman',
 'Productioncompanies': ['Walt Disney Pictures', 'Pixar Animation Studios'],
 'Distributed by': 'Walt Disney Studios Motion Pictures',
 'Release dates': ['June12,2010 ( 2010-06-12 ) ( Taormina Film Fest )',
  'June18,2010 ( 2010-06-18 ) (United States)'],
 'Running time': '103 minutes [1]',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$200million [1]',
 'Box office': '$1.067billion [1]'}

### Get info boxes from all movies




In [6]:
r= requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')

### convert to a beautiful soup object
soup = bs(r.content)

#print out HTML

contents = soup.prettify()


In [7]:
movies = soup.select(".wikitable.sortable i ")
#print(len(movies))

In [8]:
movies[0].a['title']

'Academy Award Review of Walt Disney Cartoons'

In [70]:
def get_content_value(row_data):
    if row_data.find('li'):
        return [li.get_text(' ', strip=True).replace('\xa0', '') for li in row_data.find_all('li')] ##for all list in that row
    elif row_data.find('br'):
        return[text for text in row_data.stripped_strings]
        
    else:
        return row_data.get_text(' ', strip=True).replace('\xa0', '') ##tvarkome teksta su replace

def clean_tags(soup):
    for tag in soup.find_all(['sup', 'span']):
        tag.decompose() ### nuima reference [1]

def get_info_box(url):
    
    
    r= requests.get(url)
    
    soup = bs(r.content)
    contents = soup.prettify()
    info_box = soup.find(class_='infobox venvent')
    info_rows=info_box.find_all('tr')
    
    clean_tags(soup)
    movie_info = {} 
   
    
    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title']=row.find('th').get_text(' ', strip=True) ### kad išmti nereikalingus tarpus ir sujungtus zodzius
        #elif index == 1:
            #continue
        else:
            header= row.finf('th') #neturi header info boxe
            if header:
                content_key = row.find('th').get_text(' ', strip=True)
                content_value = get_content_value(row.find('td'))
                movie_info[content_key] = content_value
        
    return movie_info


        

In [48]:
get_info_box('https://en.wikipedia.org/wiki/One_Little_Indian_(film)')

AttributeError: 'NoneType' object has no attribute 'find_all'

In [44]:
#url = 'https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films'
r = requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')
soup = bs(r.content)
wiki_url = 'https://en.wikipedia.org'
movies = soup.select('.wikitable.sortable i a')
master_movie_data = []
for index, movie in enumerate(movies):
    if index % 10 == 0:
        print(index)
    try:
        movie_title = movie['title']
        movie_url = movie['href']
        movie_data = get_info_box(wiki_url + movie_url)
        master_movie_data.append(movie_data)
    except Exception as e:
        print(movie.get_text())
        print(e)

0
Academy Award Review of Walt Disney Cartoons
'NoneType' object has no attribute 'find_all'
Snow White and the Seven Dwarfs
'NoneType' object has no attribute 'find_all'
Pinocchio
'NoneType' object has no attribute 'find_all'
Fantasia
'NoneType' object has no attribute 'find_all'
The Reluctant Dragon
'NoneType' object has no attribute 'find_all'
Dumbo
'NoneType' object has no attribute 'find_all'
Bambi
'NoneType' object has no attribute 'find_all'
Saludos Amigos
'NoneType' object has no attribute 'find_all'
Victory Through Air Power
'NoneType' object has no attribute 'find_all'
The Three Caballeros
'NoneType' object has no attribute 'find_all'
10
Make Mine Music
'NoneType' object has no attribute 'find_all'
Song of the South
'NoneType' object has no attribute 'find_all'
Fun and Fancy Free
'NoneType' object has no attribute 'find_all'
Melody Time
'NoneType' object has no attribute 'find_all'
So Dear to My Heart
'NoneType' object has no attribute 'find_all'
The Adventures of Ichabod and

True-Life Adventures
'NoneType' object has no attribute 'find_all'
Ride a Wild Pony
'NoneType' object has no attribute 'find_all'
No Deposit, No Return
'NoneType' object has no attribute 'find_all'
Treasure of Matecumbe
'NoneType' object has no attribute 'find_all'
Gus
'NoneType' object has no attribute 'find_all'
The Shaggy D.A.
'NoneType' object has no attribute 'find_all'
130
Freaky Friday
'NoneType' object has no attribute 'find_all'
The Littlest Horse Thieves
'NoneType' object has no attribute 'find_all'
The Many Adventures of Winnie the Pooh
'NoneType' object has no attribute 'find_all'
A Tale of Two Critters
'NoneType' object has no attribute 'find_all'
The Rescuers
'NoneType' object has no attribute 'find_all'
Herbie Goes to Monte Carlo
'NoneType' object has no attribute 'find_all'
Pete's Dragon
'NoneType' object has no attribute 'find_all'
Candleshoe
'NoneType' object has no attribute 'find_all'
Return from Witch Mountain
'NoneType' object has no attribute 'find_all'
The Cat f

Recess: School's Out
'NoneType' object has no attribute 'find_all'
Lady and the Tramp II: Scamp's Adventure
'NoneType' object has no attribute 'find_all'
Atlantis: The Lost Empire
'NoneType' object has no attribute 'find_all'
The Princess Diaries
'NoneType' object has no attribute 'find_all'
Max Keeble's Big Move
'NoneType' object has no attribute 'find_all'
Monsters, Inc.
'NoneType' object has no attribute 'find_all'
Mickey's Magical Christmas: Snowed in at the House of Mouse
'NoneType' object has no attribute 'find_all'
Snow Dogs
'NoneType' object has no attribute 'find_all'
Return to Never Land
'NoneType' object has no attribute 'find_all'
Cinderella II: Dreams Come True
'NoneType' object has no attribute 'find_all'
260
The Hunchback of Notre Dame II
'NoneType' object has no attribute 'find_all'
The Rookie
'NoneType' object has no attribute 'find_all'
Lilo & Stitch
'NoneType' object has no attribute 'find_all'
The Country Bears
'NoneType' object has no attribute 'find_all'
Spirited 

Tales from Earthsea
'NoneType' object has no attribute 'find_all'
Tinker Bell and the Great Fairy Rescue
'NoneType' object has no attribute 'find_all'
The Crimson Wing: Mystery of the Flamingos
'NoneType' object has no attribute 'find_all'
Secretariat
'NoneType' object has no attribute 'find_all'
Do Dooni Chaar
'NoneType' object has no attribute 'find_all'
Tangled
'NoneType' object has no attribute 'find_all'
Tron: Legacy
'NoneType' object has no attribute 'find_all'
Anaganaga O Dheerudu
'NoneType' object has no attribute 'find_all'
Beverly Hills Chihuahua 2
'NoneType' object has no attribute 'find_all'
Lilly the Witch: The Journey to Mandolan
'NoneType' object has no attribute 'find_all'
380
Mars Needs Moms
'NoneType' object has no attribute 'find_all'
Zokkomon
'NoneType' object has no attribute 'find_all'
African Cats
'NoneType' object has no attribute 'find_all'
Prom
'NoneType' object has no attribute 'find_all'
Pirates of the Caribbean: On Stranger Tides
'NoneType' object has no at

Peter Pan & Wendy
'NoneType' object has no attribute 'find_all'
Rise
'NoneType' object has no attribute 'find_all'
500
Three Men and a Baby
'NoneType' object has no attribute 'find_all'
Chang Can Dunk
'NoneType' object has no attribute 'find_all'
61
'NoneType' object has no attribute 'find_all'
Alexander and the Terrible, Horrible, No Good, Very Bad Day
'NoneType' object has no attribute 'find_all'
Atlantis: The Lost Empire
'NoneType' object has no attribute 'find_all'
citation needed
'NoneType' object has no attribute 'find_all'
Bambi
'NoneType' object has no attribute 'find_all'
Father of the Bride
'NoneType' object has no attribute 'find_all'
Haunted Mansion
'NoneType' object has no attribute 'find_all'
Hunchback
'NoneType' object has no attribute 'find_all'
510
Inspector Gadget
'NoneType' object has no attribute 'find_all'
Keeper of the Lost Cities
'NoneType' object has no attribute 'find_all'
Lilo & Stitch
'NoneType' object has no attribute 'find_all'
Muppet Man
'NoneType' object 

In [25]:


r= requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')
soup = bs(r.content)

movies = soup.select(".wikitable.sortable i a")
#print(len(movies))

base_path = 'https://en.wikipedia.org/'


movie_info_list =[]

for index, movie in enumerate(movies):
    if index == 10:
        break
    try:
        relative_path=movie['href']
        full_path = base_path + relative_path
        title= movie['title']
    
        #print(relative_path)
        #print(title)
        #print
        
        movie_info_list.append(get_info_box(full_path))
        
    except Exception as e:
        print(movie.get_text())
        print(e)




Academy Award Review of Walt Disney Cartoons
'NoneType' object has no attribute 'find_all'
Snow White and the Seven Dwarfs
'NoneType' object has no attribute 'find_all'
Pinocchio
'NoneType' object has no attribute 'find_all'
Fantasia
'NoneType' object has no attribute 'find_all'
The Reluctant Dragon
'NoneType' object has no attribute 'find_all'
Dumbo
'NoneType' object has no attribute 'find_all'
Bambi
'NoneType' object has no attribute 'find_all'
Saludos Amigos
'NoneType' object has no attribute 'find_all'
Victory Through Air Power
'NoneType' object has no attribute 'find_all'
The Three Caballeros
'NoneType' object has no attribute 'find_all'


### Save/reaload DATA

In [49]:
import json

def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [50]:
def load_data(title):
    with open(title, encoding='utf-8') as f:
        return json.load(f)

In [52]:
save_data('disney_data.json', movie_info_list)

NameError: name 'movie_info_list' is not defined

### Cleaning data

In [400]:
import pandas as pd
movie_info_list = pd.read_json('/Users/indre/Downloads/Disney-Movie-Dataset-Creation-main/disney_data.json')


In [401]:
#Convert running time into integer
#Convert dates into datetime object
#Split up the long string
#Convert budget and Box office to numbers

In [402]:
#Clean up references (remoce [1], [2] etc.)


def clean_tags(soup):
    for tag in soup.find_all(['sup', 'span']):
        tag.decompose() ### nuima reference [1]


In [403]:
#Split up long strings

def get_content_value(row_data):
    if row_data.find('li'):
        return [li.get_text(' ', strip=True).replace('\xa0', '') for li in row_data.find_all('li')] ##for all list in that row
    elif row_data.find('br'):
        return[text for text in row_data.stripped_strings] #paskutinės dvi

### CLEANED DATA

In [404]:
import pandas as pd
movie_info_list = pd.read_json('/Users/indre/Downloads/Disney-Movie-Dataset-Creation-main/disney_data_cleaned.json')


In [405]:
#Convert running time into integer '85 minutes' to 85 integer


In [406]:
print(movie_info_list.get("Running time", 'N/A')) #for movie in movie_info_list])

0      41 minutes (74 minutes 1966 release)
1                                83 minutes
2                                88 minutes
3                               126 minutes
4                                74 minutes
                       ...                 
432                             100 minutes
433                                     NaN
434                                     NaN
435                                     NaN
436                                     NaN
Name: Running time, Length: 437, dtype: object


In [407]:
import numpy as np
import pandas as pd



#'185 minutes'
def minutes_to_integer(runnig_time):
    if runnig_time == 'N/A':
        return None
    if isinstance(runnig_time, list):
        entry=runnig_time[0]
        value = int(entry.split(' ')[0])
        return value
    else: #is a string
        value = int(runnig_time.split(' ')[0])
        return value
print(minute_to_integer(['85 minutes', '90minutes']))



85


In [408]:
movie_info_list['Running time (int)'] = minutes_to_integer(movie_info_list.get('Running time', 'N/A'))


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [409]:
#Convert budget and Box office to numbers

In [410]:
print(movie_info_list.get('Budget', 'N/A'))


0                NaN
1      $1.49 million
2       $2.6 million
3      $2.28 million
4           $600,000
           ...      
432    $150 million+
433              NaN
434              NaN
435              NaN
436              NaN
Name: Budget, Length: 437, dtype: object


In [411]:
import re
amount = r'thousand|million|billion'
number = r'\d+(,\d{3})*\.*\d *'

word_re = rf'\${number}(-|\sto\s)?({number})?\s({amount})'
value_re = rf'\${number}'

def word_to_value(word):
    value_dict = {'thousand': 1000, 'million': 1000000, 'billion': 1000000000}
    return value_dict[word]


def parse_word_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(',', ' '))
    word =re.search(amount, string, flags=re.I).group().lower() #flags=re.I ignore M/m
    word_value = word_to_value(word)
    return values *word_value

def parse_value_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(',', ' '))
    return value
    

def money_conversion(money):
    if money == 'N/A':
        return None
    
    if isinstance(money, list):
        money = money[0]
        
    
    word_syntax = re.search(word_re, money, flags=re.I)
    value_syntax = re.search(value_re, money)
    
    if word_syntax:
        print ('HERE')
        return parse_word_syntax(word_syntax.group())
    elif value_syntax:
        return parse_value_syntax(value_syntax.group())
    else: 
        return None
        


In [454]:
#for movie in movie_info_list:
movie_info_list['Budget(float)'] = money_conversion(movie_info_list.get('Budget', 'N/A'))
movie_info_list['Box office(float)'] = money_conversion(movie_info_list.get('Budget', 'N/A'))
      

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [413]:
#Convert dates into datetime object

In [424]:
movie_info_list.iloc[-50]

Title                                                     The Jungle Book
Production company         [Walt Disney Pictures, Fairview Entertainment]
Release date            [April 4, 2016 ( El Capitan Theatre ), April 1...
Running time                                                  106 minutes
Country                                                     United States
Language                                                          English
Box office                                                 $966.6 million
Directed by                                                   Jon Favreau
Produced by                                 [Jon Favreau, Brigham Taylor]
Written by                                                   Justin Marks
Based on                           [The Jungle Book, by, Rudyard Kipling]
Starring                [Bill Murray, Ben Kingsley, Idris Elba, Lupita...
Music by                                                      John Debney
Distributed by                     [Wa

In [448]:
from datetime import datetime

In [480]:
dates = movie_info_list.get('Release date', 'N/A') #for movie in movie_info_list]


def clean_date(date):
    return date.split('(')[0].strip()

def date_conversion(date):
    if isinstance(date, list):
        date=date[0]
    
    if date == 'N/A':
        return None
        
        
    date_str = clean_date(date)
    print(date_str)

    fmts =['%B %d, thanks &Y', '%d %B %Y', '%d, %B %Y']
    
    for fmt in fmts:
        try:
            return datetime.strptime(date_str, fmt)
        except:
            pass
    return None
    
for date in dates:
    print(date_conversion(date))
    print()


May 19, 1937
None

December 21, 1937
None

February 7, 1940
None

November 13, 1940
None

June 20, 1941
None

October 23, 1941
None

August 9, 1942
None

August 24, 1942
None

July 17, 1943
None

December 21, 1944
None

April 20, 1946
None

November 12, 1946
None

September 27, 1947
None

May 27, 1948
None

November 29, 1948
None

October 5, 1949
None

February 15, 1950
None

June 22, 1950
None

July 26, 1951
None

March 13, 1952
None

February 5, 1953
None

August 8, 1953
None

November 10, 1953
None

26 October 1953
1953-10-26 00:00:00

August 17, 1954
None

December 23, 1954
None

May 25, 1955
None

June 22, 1955
None

September 14, 1955
None

December 22, 1955
None

June 8, 1956
None

July 18, 1956
None

November 6, 1956
None

December 20, 1956
None

June 19, 1957
None

August 28, 1957
None

December 25, 1957
None

July 9, 1958
None

August 12, 1958
None

December 25, 1958
None

January 29, 1959
None

March 19, 1959
None

June 24, 1959
None

November 10, 1959
None

January 21, 1960

AttributeError: 'float' object has no attribute 'split'

In [498]:
for movie in movie_info_list:
    movie['Release date (datetime)'] = date_conversion(movie.get('Release date', 'N/A'))

AttributeError: 'str' object has no attribute 'get'

In [499]:
### MORE CLEANED DATA 

In [2]:
import pandas as pd
import pickle
import base64
import csv

movie_info_list = pd.read_pickle('/Users/indre/Downloads/Disney-Movie-Dataset-Creation-main/disney_data_cleaned_more.pickle')

movie_info_list = pd.DataFrame(movie_info_list)




In [3]:
movie_info_list.iloc[7]

Title                                                         Saludos Amigos
Production company                                   Walt Disney Productions
Release date               [August 24, 1942 (World Premiere-Rio de Janeir...
Running time                                                      42 minutes
Country                                                        United States
Language                                      [English, Portuguese, Spanish]
Box office                                    $1,135,000 (worldwide rentals)
Running time (int)                                                      42.0
Budget (float)                                                           NaN
Box office (float)                                                 1135000.0
Release date (datetime)                                  1942-08-24 00:00:00
Directed by                [Norman Ferguson, Wilfred Jackson, Jack Kinney...
Produced by                                                      Walt Disney

### Attach IMBD/Rotten Tomatoes scores
#### getting imdb scrore from http://www.omdbapi.com/apikey.aspx

In [11]:
import requests
import urllib
import os 

def get_omdb_info(title):
    base_url =  'http://www.omdbapi.com/?'
    parameters = {'apikey': '5945f9c7 ', 't':title}
    params_encoded = urllib.parse.urlencode(parameters)
    full_url = base_url + params_encoded
    return requests.get(full_url).json()

def get_rotten_tomato_score(omdb_info):
    ratings = omdb_info.get('Ratings', [])
    for rating in ratings:
        if rating['Source'] == 'Rotten Tomatoes':
            return rating['Value']
    return None
        
info =  get_omdb_info('into the woods')
get_rotten_tomato_score(info)
    

'71%'

In [29]:
for movie in movie_info_list:
    title = movie['title']
    omdb_info = get_omdb_info(title)
    movie['imdb'] =omdb_info.get('imdbRating', None)
    movie['metascore'] = omdb_info.get('Metascore', None)
    movie['rotten_tomatoes'] = get_rotten_tomato_score(omdb_info)


TypeError: string indices must be integers

In [ ]:
### FINAL CLEAN



In [35]:
movie_info_list = pd.read_csv('/Users/indre/Downloads/Disney-Movie-Dataset-Creation-main/disney_data_final 2.csv')
movie_info_list.head(3)


,Unnamed: 0,Title,Production company,Release date,Running time,Country,Language,Box office,Running time (int),Budget (float),...,Narrated by,Cinematography,Edited by,Screenplay by,Production companies,Japanese,Hepburn,Adaptation by,Traditional,Simplified
0,0,Academy Award Review of,Walt Disney Productions,"['May 19, 1937']",41 minutes (74 minutes 1966 release),United States,English,$45.472,41.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Snow White and the Seven Dwarfs,Walt Disney Productions,"['December 21, 1937 ( Carthay Circle Theatre ,...",83 minutes,United States,English,$418 million,83.0,1490000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Pinocchio,Walt Disney Productions,"['February 7, 1940 ( Center Theatre )', 'Febru...",88 minutes,United States,English,$164 million,88.0,2600000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
movie_info_list.iloc[5]

Unnamed: 0                                                                 5
Title                                                                  Dumbo
Production company                                   Walt Disney Productions
Release date               ['October 23, 1941 (New York City)', 'October ...
Running time                                                      64 minutes
Country                                                        United States
Language                                                             English
Box office                 $1.3 million (est. United States/Canada rental...
Running time (int)                                                      64.0
Budget (float)                                                      950000.0
Box office (float)                                                 1300000.0
Release date (datetime)                                           1941-10-23
IMDB                                                                     7.2